# AI-Gore for learning SDGs

This is an app inspired by a famous LLM-based mobile game called Infinite Craft. 

The UI of the Infinite Craft is very simple. All it does is to enable the player to combine two words (at the beginning only four words - earth, wind, fire, and water- can be combined with each other) such as "stone" and "fire" to generate a new word "lava".

While the Infinite Craft is played in many ways, the most common way to play this with friends is to compete how many steps you needed to reach to the target word. For example how many words will be needed to obtain the word "Vienna".

To excell in this game, one need to know the common words associated to the target word. I.e., for "Vienna" we might consider words such as "Waltz" or "Music" to be highly associated.

For this reason, we decided to build the Infinite Craft game in the context of SDGs so that someone unfamiliar with this concept can better develop some understanding of it in a playful manner. 

The inspiration for the name AI-Gore is obvious.

In [ ]:
# !pip -q install pandas squarify langchain docx2txt pypdf bitsandbytes accelerate xformers einops datasets loralib sentencepiece sentence_transformers chromadb

Run the code above in case you haven't run the UnidoMain.ipynb file yet where there is a same chunk that downloads all relevant packages called in the chunk below

In [117]:
import pandas as pd
import torch
import gc
import os

from langchain.document_loaders import PyPDFLoader,Docx2txtLoader
from langchain.text_splitter import RecursiveCharacterTextSplitter
from langchain.embeddings import HuggingFaceEmbeddings
from langchain.vectorstores import Chroma
from langchain.chains import RetrievalQA
from transformers import AutoTokenizer, AutoModelForCausalLM, pipeline,BitsAndBytesConfig
from langchain.chains.combine_documents.stuff import StuffDocumentsChain
from langchain_community.llms import HuggingFacePipeline
from langchain.prompts import PromptTemplate
from langchain.chains.llm import LLMChain
from langchain.chat_models import ChatOpenAI
from langchain_community.llms import DeepInfra
from langchain.chains.combine_documents import create_stuff_documents_chain

Main prompt

In [126]:
prompt = """
// Lines in this file starting with '//' are comments and will be ignored
// The following is a basic system prompt to run the game using the Alpaca template
// Variables:
// - {question} - The user's question
// - {context} - The user's context 
Below is an instruction that describes a task, paired with an question and the context. Write a response that appropriately completes the request.
 
### Instruction:
We are going to play a game called twisted version of "Infinite Craft". Infinite Craft is a game like Little Alchemy but you can combine ANY element to possibly get a output!
The inputs to this game are the two words to be combined with.
The output will either be another word closely associated with the two words, or one of the 17 Sustanable Developments Goals (SDGs) provided in the context if the combined meaning of these two words are sufficiently close to one of it. 
 
For example, combining "Hunger" and "Eradicate" might give us the second SDGs "Zero Hunger". 

While combining "World" and "Eradicate" will not relate to any of the 17 SDGs, so you give us instead a word "Apocalypse" 
which is closest to the combined meaning of two words "World" and "Eradicate". Be creative with your answer.

See below for the example input and output.

Examples:
 
- Input: "Hunger, Eradicate"
- Output:
SDG 2: Zero Hunger
```
 
- Input: "World, Eradicate"
- Output:
Apocalypse
```
 
Sometimes the user will combine two of the same element. In some cases the output should be a bigger version of the element. For example, combining "Urban" and "Urban" might give us "Metropolis".

Below is the list of SDGs: 

*** IMPORTANT ***
NEVER output more then 1 word at a time unless it's one of the SDGs.
Make sure to be creative but still logically correct!!!
The response MUST be simple and relatively short.

### Context:
{context}


### Question:
{question}
 
### Response:
"""

Make sure the SDGs.pdf is in the same directory. This is used as source in the RAG.

In [127]:
document =[]
loader = PyPDFLoader("SDGs.pdf")
document.extend(loader.load())
document_splitter = RecursiveCharacterTextSplitter(separators = ["\n\n","\n"," ", ""],
                                                   chunk_size = 1000,
                                                   chunk_overlap = 200,
                                                   length_function = len)
document_chunks = document_splitter.split_documents(document)
embeddings = HuggingFaceEmbeddings(model_name = 'sentence-transformers/all-MiniLM-L6-v2')  
vectordb = Chroma.from_documents(document_chunks, embedding = embeddings)

/Users/jan/opt/anaconda3/lib/python3.11/site-packages/torch/_utils.py:776: UserWarning: TypedStorage is deprecated. It will be removed in the future and UntypedStorage will be the only storage class. This should only matter to you if you are using storages directly.  To access UntypedStorage directly, use tensor.untyped_storage() instead of tensor.storage()
  return self.fget.__get__(instance, owner)()


Below is the API token to the LLama model we used through DeepInfra

In [129]:
os.environ["DEEPINFRA_API_TOKEN"] = "zAMiCKmaweiQ0nNM0jMaV956QXXjTF1a"

In [131]:
llm = DeepInfra(model_id="meta-llama/Llama-2-70b-chat-hf")
llm.model_kwargs = {
    "temperature": 0.2,
    "repetition_penalty": 1.2,
    "max_new_tokens": 250,
    "top_p": 0.9,
}

In [132]:
# RAG part
QA_CHAIN_PROMPT = PromptTemplate.from_template(prompt)
llm_chain = LLMChain(llm=llm, prompt=QA_CHAIN_PROMPT, callbacks=None, verbose=False)
# combine documents retrieved from vector database
document_prompt = PromptTemplate(
    input_variables=["page_content", "source"],
    template="Context:\ncontent:{page_content}\nsource:{source}",
)
combine_documents_chain = StuffDocumentsChain(
        llm_chain=llm_chain,
        document_variable_name="context",
        document_prompt=document_prompt,
        callbacks=None,
)
# feed all above into this function to carry out the QA
qa_chain = RetrievalQA(
    combine_documents_chain=combine_documents_chain,
    callbacks=None,
    verbose=False,
    retriever = vectordb.as_retriever(search_kwargs={'k':5}),
    return_source_documents = False
)

The UI works in the following manner:

* At the beginning, one word is randomly drawn and is treated as the Base Word

* The player will choose another word from the dropdown menu and the combined word will be the Base Word for the next try

* After the 10th try, if the player is still unable to get to one of the SDGs, the game will stop

Sometimes, the LLM makes a very awkward word such as "Education Empire" which is quite amusing. 

On the other hand, we also observed many occasions in which it tries to associate words that does not even remotely seem to be associated to a particular SDG as the combined word, so still a lot of work is needed. 

In [135]:
import ipywidgets as widgets
from IPython.display import display
import random
# Initialize counter
counter = 10

list_of_words = ['Poverty', 'Gender','Female','Hunger','Health','Education',
             'Sanitation','Energy','Economy','Development','Work',
             'Industry','Infrastructure','Digital','Consumption',
            'Production','Climate','Ocean','Forest','Soil','Animal',
            'Justice','Inequality','Fairness','Collaboration','Urban',
            'Administrative','Cheap','Accessible','Abundant','Decrease',
            'Empower','Objective','Group','Team','Reliable',"Eradicate",
            "Nullify",'Being','World','Global']
# Dropdown options
options_1 = random.choice(list_of_words)
options_2 = list_of_words

# Dropdown widgets
dropdown_1 = widgets.Dropdown(options=options_1, description='Base Word')
dropdown_1.options = [options_1]
dropdown_2 = widgets.Dropdown(options=options_2, description='Chosen Word')

# Button widget
button = widgets.Button(description="Enter")

# Counter widget
counter_display = widgets.Label(value=f'Counter: {counter}')

# Message widget
message = widgets.Label()

# Result widget
result = widgets.Label()

# Function to handle button click event
def on_button_clicked(b):
    global counter
    if dropdown_1.value is not None and dropdown_2.value is not None:
        if counter > 0:
            counter -= 1
            counter_display.value = f'Counter: {counter}'
            combined_word = qa_chain(dropdown_1.value + ", " + dropdown_2.value)['result']
            result.value = f'Combined word: {combined_word}'
            dropdown_1.options = [combined_word]  # Update dropdown options
            if counter == 0:
                button.disabled = True
                message.value = "Limit reached. You can no longer click the button."
        else:
            message.value = "Limit reached. You can no longer click the button."
    else:
        message.value = "Please select options from both dropdowns before clicking the button."

button.on_click(on_button_clicked)

# Display UI
display(dropdown_1)
display(dropdown_2)
display(button)
display(counter_display)
display(message)
display(result)

Dropdown(description='Base Word', options=('Global',), value='Global')

Dropdown(description='Chosen Word', options=('Poverty', 'Gender', 'Female', 'Hunger', 'Health', 'Education', '…

Button(description='Enter', style=ButtonStyle())

Label(value='Counter: 10')

Label(value='')

Label(value='')